In [1]:
# 15-Sep-2019
# Bo Milvang-Jensen
# Very rudimentary script to read+plot ...

import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt

# Read photometry catalogue created by my other script
photfilename = 'F51_out_REMIR_ROS2.fits'
data = fits.getdata(photfilename, 1)

# The photometry catalogue contains aperture photometry in 25 different
# apertures. If we number them 0..24, they correspond to diameters 1..25 arcsec
apno = 10 # This is then 11 arcsec

# The photometry catalogue contains photometry for the AGN plus some more
# objects in the field, in this case the AGN + 6 more objetcs, so 7 in
# total. The AGN is number 1, and the others 2, 3, etc.
# This is seen in the column names, which end in _1, _2, etc.
# E.g. the column MAG_APER_1 contains the aperture magnitudes (for all
# 25 apertures) for the AGN.

# Get the aperture magnitudes
##foo = data['MAG_APER_1'] # has shape e.g. (767, 25), the 25 being the apertures
mag_agn  = data['MAG_APER_1'][:,apno]
mag_ref1 = data['MAG_APER_3'][:,apno] # I have found that _3 is good here
mag_ref2 = data['MAG_APER_4'][:,apno]

# Get the differential magnitudes
delta_mag_agn_ref1 = mag_agn - mag_ref1
delta_mag_ref2_ref1 = mag_ref2 - mag_ref1

# Get the magnitude errors 
magerr_agn = data['MAGERR_APER_1'][:,apno]
magerr_ref1 = data['MAGERR_APER_3'][:,apno]
magerr_ref2= data['MAGERR_APER_4'][:,apno]

# TODO Here I could loop over the filters, subtracting the median

# For simplicity "extract" some arrays from the data
filt = data['filter']
mjd_obs = data['mjd_obs']

# A simple plot of the J band differential mags, with the median subtracted
# Do 'H' and 'K' in the same way
mask = (filt == 'J')
#print(magerr_ref1[mask])
#plt.scatter(mjd_obs[mask], delta_mag_agn_ref1[mask]-np.median(delta_mag_agn_ref1[mask]))
plt.errorbar(mjd_obs[mask], delta_mag_agn_ref1[mask]-np.median(delta_mag_agn_ref1[mask]), magerr_ref1[mask],fmt='b.')
plt.title('test')
plt.xlabel('MJD-OBS')
plt.ylabel('delta mag')
plt.show()

<Figure size 640x480 with 1 Axes>

In [2]:
#J band data
xJ=np.asarray(mjd_obs[mask], dtype=float)
nJ=len(xJ)
XJ = np.reshape(xJ,(nJ,1))
yJ=np.asarray(delta_mag_agn_ref1[mask]-np.median(delta_mag_agn_ref1[mask]), dtype=float)
yJerr = np.array(magerr_ref1[mask], dtype=float)


#K band data
mask2 = (filt == 'K')
xK=np.asarray(mjd_obs[mask2], dtype=float)
nK=len(xK)
XK = np.reshape(xK,(nK,1))
yK=np.asarray(delta_mag_agn_ref1[mask2]-np.median(delta_mag_agn_ref1[mask2]), dtype=float)
yKerr = np.asarray(magerr_ref1[mask2], dtype=float)
print(nJ)

49


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import theano
import theano.tensor as tt
from theano.tensor import fft

X_new_temp,muJ_temp,sdJ_temp = np.loadtxt('Jband.txt', delimiter=',', usecols=(0,1,2), unpack=True)
XK_new_temp,muK_temp,sdK_temp = np.loadtxt('Kband.txt', delimiter=',', usecols=(0,1,2), unpack=True)

i_low=1
i_high=100
X_new=X_new_temp[i_low:i_high]
X_new=np.reshape(X_new,(len(X_new),1))
muJ=muJ_temp[i_low:i_high]
sdJ=sdJ_temp[i_low:i_high]

XK_new=XK_new_temp[i_low:i_high]
muK=muK_temp[i_low:i_high]
sdK=sdK_temp[i_low:i_high]

tau=np.linspace(1,100,99)#time delay array
tau=np.reshape(tau,(1,1,99,1))

print(X_new.shape)

(99, 1)


In [5]:
import theano
import theano.tensor as tt
import theano.tensor.signal.conv

with pm.Model() as convmodel:
    
    #define driving function as Gaussian Process
    #fine way to use g band as first guess of value 
    ℓ = pm.Uniform('ℓ', lower=1.2574, upper=6.0*50.0*np.sqrt(2.0))#
    η = pm.Uniform('η', lower=0.0, upper=1.0)#
    cov = η**2 * pm.gp.cov.Exponential(1, ℓ)#using same cov as light curve interpolation
    gp = pm.gp.Latent(cov_func=cov)
    f = gp.prior("f", X=X_new)#use the g band time values here
    f = f.reshape((1,1,len(X_new),1))
    #print(f.shape.ndim)
    #print(f.tag.test_value)
    
    
    #Define priors
    sigma_DT=pm.Uniform('sigma_DT', lower=tt.log(2.0), upper=tt.log(500.0))#needs a source for scale
    sigma_AD=pm.Uniform('sigma_AD', lower=tt.log(2.0), upper=tt.log(500.0))#needs a source for scale
    mu_DT=pm.Uniform('mu_DT', lower=tt.log(10.0), upper=tt.log(1000.0))#we expect serveral tens to hundreds of days
    mu_AD=pm.Uniform('mu_AD', lower=tt.log(2.0), upper=tt.log(300.0))#AD has 3-5 times smaller lags than DT 
    A_T=pm.Uniform('A_T', lower=0.0, upper=1.0)#needs to be restricted between 0 to 1
    wav=pm.Uniform('wav', lower=tt.log(100.0), upper=tt.log(10000.0))#the range is large 
    T=pm.Uniform('T', lower=tt.log(1000.0), upper=tt.log(1700.0))#try a normal here instead
    K_0=pm.Uniform('K_0', lower=0.0, upper=10.0)#is it BB/powr or powr/BB?
    index=pm.Uniform('index', lower=1.0, upper=2.0)#sign depends on diffmag definition change to -2 to -1 for final

    #Define constants 
    wav_0 = 5000.0#Reference wavelength
    h = 6.626e-34#Plancks constant
    c = 299792458.0#speed of light
    k = 1.38e-23#Boltzmanns constant
    
    #Dusty Torus transfer equation
    wav_peak = 2.898*10**6/tt.exp(T)
    b_max = h*c/(1e-9*wav_peak*k*tt.exp(T))
    BB_max = 1.0/( (wav_peak**5) * (tt.exp(b_max) - 1.0) )
    b = h*c/(1e-9*tt.exp(wav)*k*tt.exp(T))
    BB = (1.0/( (tt.exp(wav)**5) * (tt.exp(b) - 1.0) ))/BB_max

    exp_DT = -((tt.log(tau)-mu_DT)**2/(2*sigma_DT**2))
    front_DT = A_T/(tau*sigma_DT*np.sqrt(2*np.pi))
    lognorm_DT = front_DT*np.exp(exp_DT)
    Psi_DT = BB*lognorm_DT#/lognorm_DT.sum(axis=2)#/DT_max#
    
    #Accretion Disk transfer equation
        
    powr = K_0*(tt.exp(wav)/wav_0)**(index)    

    exp_AD = -((tt.log(tau)-mu_AD)**2/(2*sigma_AD**2))
    front_AD = (1.0-A_T)/(tau*sigma_AD*np.sqrt(2*np.pi))
    lognorm_AD = front_AD*tt.exp(exp_AD)
    Psi_AD = powr*lognorm_AD#/lognorm_AD.sum(axis=2)#/AD_max#

    
    #Full transfer equation
    transfer = (Psi_DT + Psi_AD)
    
    #The convolution
    muJT=np.reshape(muJ,(1,1,99,1))#define muJ the right way
    
    convol=theano.tensor.nnet.conv2d(f,transfer,border_mode='half')
    #print(len(convol[0,0,:,0].tag.test_value))
    
    #Define likelihood
    likelihood = pm.Normal('muK', mu=convol[0,0,:,0], sigma=sdK, observed=muK)#include noise boost factor again

    tracetransfer = pm.sample(2000, tune=1000)#tuning improves speed?

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
C:\Users\Ejer\Anaconda3\lib\site-packages\theano\tensor\basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
C:\Users\Ejer\Anaconda3\lib\site-packages\theano\tensor\basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [index, K_0, T, wav, A_T, mu_AD, mu_DT, sigma_AD, sigma_DT, f_rotated_, η, ℓ]


KeyboardInterrupt: 